自定义的输出规则,让LLM不仅只是文本聊天,还可以与现实各种系统无缝对接

In [ ]:
# 讲笑话机器人：希望每次根据指令，可以输出一个这样的笑话(小明是怎么死的？笨死的)
from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel,Field,validator
from typing import List
import os

api_base = os.getenv("OPENAI_PROXY")
api_key = os.getenv("OPENAI_API_KEY")

# 构造LLM
model = OpenAI(
    model = "gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key = api_key,
    openai_api_base = api_base,
)

# 定义个数据模型，用来描述最终的实例结构
class Joke(BaseModel):
    setup:str = Field(description="设置笑话的问题")
    punchline:str = Field(description="回答笑话的答案")

    #验证问题是否符合要求
    @validator("setup")
    def question_mark(cls, field):
        if field[-1] != "？":
            raise ValueError("不符合预期的问题格式!")
        return field

# 将Joke数据模型传入
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template = "回答用户的输入.\n{format_instructions}\n{query}\n",
    input_variables = ["query"],
    partial_variables = {"format_instructions":parser.get_format_instructions()}
)

prompt_and_model = prompt | model
out_put = prompt_and_model.invoke({"query":"给我讲一个笑话"})
print("out_put:",out_put)
parser.invoke(out_put)

In [ ]:
# LLM的输出格式化成python list形式，类似['a','b','c']
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
import os

api_base = os.getenv("OPENAI_PROXY")
api_key = os.getenv("OPENAI_API_KEY")

# 构造LLM
model = OpenAI(
    model = "gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key = api_key,
    openai_api_base = api_base,
)

# 输出的数据模型
parser = CommaSeparatedListOutputParser()

prompt = PromptTemplate(
    template = "列出5个{subject}.\n{format_instructions}",
    input_variables = ["subject"],
    partial_variables = {"format_instructions":parser.get_format_instructions()}
)

_input = prompt.format(subject="常见的小狗的名字")
output = model(_input)
print(output)
#格式化
parser.parse(output)